In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**I. Load data**

In [21]:
import os
from tqdm import tqdm
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

In [3]:
gray_image_folder = '/content/drive/MyDrive/gray_SAR'  
color_image_folder = '/content/drive/MyDrive/color_SAR'  

In [18]:
# Create lists of image paths
gray_image_paths = [os.path.join(gray_image_folder, filename) for filename in os.listdir(gray_image_folder)]
color_image_paths = [os.path.join(color_image_folder, filename) for filename in os.listdir(color_image_folder)]

In [19]:
# Functions to convert images
def convert_image_labels(images):
    labels = []
    for img_path in tqdm(images):
        i = cv2.imread(img_path)
        if i is not None:
            i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
            res_i = cv2.resize(i, (SIZE, SIZE))
            labels.append(res_i)
    return labels

def convert_image_inputs(images):
    inputs = []
    for img_path in tqdm(images):
        z = cv2.imread(img_path)
        if z is not None:
            z = cv2.cvtColor(z, cv2.COLOR_BGR2GRAY)
            res_z = cv2.resize(z, (SIZE, SIZE))
            inputs.append(res_z)
    return inputs

In [22]:
# Convert the images using the defined paths
labels = convert_image_labels(color_image_paths)  # Use color images for labels
inputs = convert_image_inputs(gray_image_paths)    # Use grayscale images for inputs

100%|██████████| 700/700 [00:11<00:00, 63.28it/s]


In [4]:
SIZE = 64

#**3. Convert data into input and label sets**

In [23]:
# Function to load images from a folder
def load_images_from_folder(folder, size, color_mode='gray'):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if color_mode == 'gray':
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load grayscale images
        else:
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Load color images (in BGR format)

        if img is not None:
            img = cv2.resize(img, (size, size))  # Resize image to (SIZE, SIZE)
            if color_mode == 'gray':
                img = np.expand_dims(img, axis=-1)  # Add channel dimension to grayscale images
            images.append(img)
    return np.array(images)

In [24]:
# Load grayscale and color images
gray_images = load_images_from_folder(gray_image_folder, SIZE, color_mode='gray')
color_images = load_images_from_folder(color_image_folder, SIZE, color_mode='color')

In [25]:
# Normalize the images to range [0, 1]
gray_images = gray_images / 255.0
color_images = color_images / 255.0

In [26]:
assert gray_images.shape[0] == color_images.shape[0], "Mismatch in number of grayscale and color images!"

In [27]:
x_train, x_test, y_train, y_test = train_test_split(gray_images, color_images, test_size=0.2, random_state=1)

In [28]:
print(f'Training set shape: {x_train.shape}, {y_train.shape}')
print(f'Test set shape: {x_test.shape}, {y_test.shape}')

Training set shape: (560, 128, 128, 1), (560, 128, 128, 3)
Test set shape: (140, 128, 128, 1), (140, 128, 128, 3)


#**Build model GAN-Unet**

##**1. Build GENERATOR(Unet)**

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import (
    BatchNormalization, Conv2D, Conv2DTranspose, Dense,
    Flatten, Dropout, UpSampling2D, Concatenate, ELU,
    Input, LeakyReLU, MaxPooling2D, Reshape, UpSampling2D)
from tensorflow.keras.models import Model
import tensorflow.keras.layers as tfl
from tensorflow.keras.optimizers import Adam

In [12]:
def build_generator(inputsize=(SIZE,SIZE,1)):
    input = tf.keras.Input(shape=(inputsize))

    conv = tfl.Conv2D(64, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv1')(
        input)
    x=tfl.BatchNormalization()(conv)
    x=tfl.LeakyReLU()(x)

    x = tfl.Conv2D(64, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv2')(
        x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)

    x1 = tfl.Conv2D(64, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv3')(
        x)
    x=tfl.BatchNormalization()(x1)
    x=tfl.LeakyReLU()(x)
    x = tfl.MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='MaxPool1')(x)

    x = tfl.Conv2D(128, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv4')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(128, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv5')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x2 = tfl.Conv2D(128, (3, 3), padding="same", strides=(1, 1),kernel_initializer='he_normal',
                    name='Conv6')(x)
    x=tfl.BatchNormalization()(x2)
    x=tfl.LeakyReLU()(x)
    x = tfl.MaxPool2D(pool_size=(2, 2), name='MaxPool2')(x)

    x = tfl.Conv2D(256, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv7')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(256, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv8')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x3 = tfl.Conv2D(256, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv9')(x)
    x=tfl.BatchNormalization()(x3)
    x=tfl.LeakyReLU()(x)
    x = tfl.MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='MaxPool3')(x)


    x = tfl.Conv2D(512, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv10')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(512, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv11')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x4 = tfl.Conv2D(512, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv12')(x)
    x=tfl.BatchNormalization()(x4)
    x=tfl.LeakyReLU()(x)
    x = tfl.MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='MaxPool4')(x)

    x = tfl.Conv2D(1024, (3, 3), padding="same",name='Conv13')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(1024, (3, 3), padding="same",name='Conv14')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(1024, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv15')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2DTranspose(512, (3, 3), strides=2, padding="same")(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)

    x = tfl.concatenate([x, x4], axis=3)

    x = tfl.Conv2D(512, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv16')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(512, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv17')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(512, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv18')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2DTranspose(256, (3, 3), strides=2, padding="same")(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)

    x = tfl.concatenate([x, x3], axis=3)

    x = tfl.Conv2D(256, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv19')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(256, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv20')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(256, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv21')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2DTranspose(128, (3, 3), strides=2, padding="same")(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)

    x = tfl.concatenate([x, x2], axis=3)

    x = tfl.Conv2D(128, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv22')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(128, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv23')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(128, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv24')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2DTranspose(64, (3, 3), strides=2, padding="same")(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)

    x = tfl.concatenate([x, x1], axis=3)

    x = tfl.Conv2D(64, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv25')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(64, (3, 3), padding="same",kernel_initializer='he_normal', name='Conv26')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    x = tfl.Conv2D(64, (3, 3), padding="same",name='Conv27')(x)
    x=tfl.BatchNormalization()(x)
    x=tfl.LeakyReLU()(x)
    # outputs = tfl.Conv2D(3, (1, 1), padding="same", activation='sigmoid', name='Outputs')(x)
    # final_model = tf.keras.Model(inputs=input, outputs=outputs)
    outputs = tfl.Conv2D(3, (1, 1), padding="same", activation='tanh', name='Generator_Output')(x)

    generator_model = tf.keras.Model(inputs=input, outputs=outputs)
    generator_model.summary()
    return generator_model

In [13]:
generator = build_generator(inputsize=(SIZE, SIZE, 1))

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 64, 1)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 64, 64, 64)     │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 64, 64, 64)     │            256 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu (LeakyReLU)   │ (None, 64, 64, 64)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv2 (Conv2D)            │ (None, 64, 64, 64)     │         36,928 │ leaky_re_lu[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 64, 64, 64)     │            256 │ Conv2[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_1 (LeakyReLU) │ (None, 64, 64, 64)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv3 (Conv2D)            │ (None, 64, 64, 64)     │         36,928 │ leaky_re_lu_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 64, 64, 64)     │            256 │ Conv3[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_2 (LeakyReLU) │ (None, 64, 64, 64)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ MaxPool1 (MaxPooling2D)   │ (None, 32, 32, 64)     │              0 │ leaky_re_lu_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv4 (Conv2D)            │ (None, 32, 32, 128)    │         73,856 │ MaxPool1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 32, 32, 128)    │            512 │ Conv4[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_3 (LeakyReLU) │ (None, 32, 32, 128)    │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv5 (Conv2D)            │ (None, 32, 32, 128)    │        147,584 │ leaky_re_lu_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4     │ (None, 32, 32, 128)    │            512 │ Conv5[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ leaky_re_lu_4 (LeakyReLU) │ (None, 32, 32, 128)    │              0 │ batch_normalization_4… │
├──────────────────────

 Total params: 50,258,499 (191.72 MB)

 Trainable params: 50,238,915 (191.65 MB)

 Non-trainable params: 19,584 (76.50 KB)

In [14]:
SIZE = 128

In [ ]:
import cv2
from skimage.color import rgb2lab, lab2rgb

def convert_image_labels(images):
    labels=[]
    for i in tqdm(images):
        i = cv2.imread(i)
        i=cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
        res_i=cv2.resize(i,(SIZE,SIZE))
        del i
        labels.append(res_i)
    return labels


def convert_image_inputs(images):
    inputs=[]
    for z in tqdm(images):
        z = cv2.imread(z)
        z=cv2.cvtColor(z,  cv2.COLOR_BGR2GRAY)
        res_z=cv2.resize(z,(SIZE,SIZE))
        del z
        inputs.append(res_z)
    return inputs

labels=convert_image_labels(images)
inputs=convert_image_inputs(images)

In [ ]:
def convert_image_inputs(images):
    labels = []
    for image_path in images:
        img = cv2.imread(image_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            res_img = cv2.resize(img, (SIZE, SIZE))
            labels.append(res_img)
    return labels
labels=convert_image_labels(images)


In [ ]:
def images_compare(inputs,labels):
    idx_new=np.random.randint(0,len(labels))
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(inputs[idx_new],cmap='gray')
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(labels[idx_new])
    plt.show()
images_compare(inputs,labels)

#**4. Split data to Train/Test set**


In [ ]:
from sklearn.model_selection import train_test_split

def split_data(inputs,labels,test_size=0.2):
    labels=np.array(labels)
    inputs=np.array(inputs)
    x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=test_size)
    return x_train, x_test ,y_train,y_test

In [ ]:
x_train,x_test ,y_train, y_test=split_data(inputs,labels,test_size=0.2)
x_train, x_test=x_train/255.0,x_test/255.0
y_train, y_test=y_train/255.0,y_test/255.0

# 

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.layers import BatchNormalization, Conv2D, Conv2DTranspose, Dense, Flatten, Dropout, UpSampling2D, Concatenate, ELU, Input, LeakyReLU, MaxPooling2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

##**2. Build DISCRIMINATOR(CNN)**

In [ ]:
generator = build_generator(inputsize=(SIZE, SIZE, 1))

In [ ]:
def build_discriminator():

    activation = tf.keras.layers.LeakyReLU(alpha=0.2)  # Sử dụng LeakyReLU như Model 1

    inputs = Input(shape=(SIZE, SIZE, 3))  # Điều chỉnh kích thước đầu vào để phù hợp với Model 1

    # Thay đổi các lớp Conv2D để có số lớp và kích thước đầu ra tương tự
    conv1 = Conv2D(64, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = activation(conv1)
    conv1 = Conv2D(64, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = activation(conv1)
    conv1 = MaxPooling2D()(conv1)

    conv2 = Conv2D(128, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = activation(conv2)
    conv2 = Conv2D(128, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = activation(conv2)
    conv2 = MaxPooling2D()(conv2)

    conv3 = Conv2D(256, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv2)
    conv3 = BatchNormalization()(conv3)
    conv3 = activation(conv3)
    conv3 = Conv2D(256, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = activation(conv3)
    conv3 = MaxPooling2D()(conv3)

    conv4 = Conv2D(512, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv3)
    conv4 = BatchNormalization()(conv4)
    conv4 = activation(conv4)
    conv4 = Conv2D(512, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = activation(conv4)
    conv4 = MaxPooling2D()(conv4)

    conv5 = Conv2D(1024, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv4)
    conv5 = BatchNormalization()(conv5)
    conv5 = activation(conv5)
    conv5 = Conv2D(1024, kernel_size=(3, 3), strides=1, padding='same', kernel_initializer='he_normal')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = activation(conv5)

    fully = Flatten()(conv5)
    fully = Dense(512)(fully)
    fully = BatchNormalization()(fully)
    fully = activation(fully)
    fully = Dense(128)(fully)
    fully = BatchNormalization()(fully)
    fully = activation(fully)
    fully = Dense(32)(fully)
    fully = BatchNormalization()(fully)
    fully = activation(fully)
    fully = Dense(1, activation='sigmoid')(fully)

    model = tf.keras.models.Model(inputs, fully)
    model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
    model.summary()
    return model


In [ ]:
discriminator = build_discriminator()

In [ ]:
discriminator = build_discriminator()

##**3. Build a complete GAN model from the two Generator and Discriminator models above**

In [ ]:
def build_gan(generator, discriminator):
    discriminator.trainable = False
    input = tf.keras.Input(shape=(SIZE, SIZE, 1))
    generated_image = generator(input)
    validity = discriminator(generated_image)
    gan = tf.keras.Model(inputs=input, outputs=[generated_image, validity])
    gan.compile(loss=['mse', 'binary_crossentropy'], loss_weights=[0.9999, 0.0001], optimizer='adam')
    gan.summary()
    return gan

gan = build_gan(generator, discriminator)


#**Train model**

In [ ]:
import numpy as np
import time

def train_gan(generator, discriminator, gan, x_train, y_train, epochs=1000, batch_size=64):
    batch_count = x_train.shape[0] // batch_size
    d_loss_list = []  # List to save discriminator loss
    g_loss_list = []  # List to save generator loss
    d_accuracy_list = []  # List to save discriminator accuracy

    for e in range(epochs):
        start_time = time.time()  # Start time

        d_acc_epoch = []  # Accuracy for each batch in the epoch

        for _ in range(batch_count):
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            real_images = y_train[idx]
            gray_images = x_train[idx]

            fake_images = generator.predict(gray_images)

            valid = np.ones((batch_size, 1))
            fake = np.zeros((batch_size, 1))

            # Train discriminator
            d_loss_real, d_acc_real = discriminator.train_on_batch(real_images, valid)
            d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_images, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            d_acc = 0.5 * np.add(d_acc_real, d_acc_fake)

            # Save discriminator accuracy for the epoch
            d_acc_epoch.append(d_acc)

            # Train generator
            valid = np.ones((batch_size, 1))
            g_loss = gan.train_on_batch(gray_images, [real_images, valid])

        # Calculate average accuracy for the epoch
        avg_d_acc = np.mean(d_acc_epoch)

        end_time = time.time()  # End time
        elapsed_time = end_time - start_time

        print(f"Epoch {e}/{epochs}, D Loss: {d_loss[0]}, G Loss: {g_loss[0]}, D Accuracy: {avg_d_acc:.2f}, Time: {elapsed_time:.2f} seconds")

        # Save losses and accuracy
        d_loss_list.append(d_loss[0])
        g_loss_list.append(g_loss[0])
        d_accuracy_list.append(avg_d_acc)

        # Compute remaining time
        remaining_time = (epochs - e - 1) * elapsed_time
        hours = int(remaining_time // 3600)
        minutes = int((remaining_time % 3600) // 60)
        print(f"Estimated time remaining: {hours} hours {minutes} minutes")

    return d_loss_list, g_loss_list, d_accuracy_list


In [ ]:
num_epochs = 200
batch_size = 32

# Call the train_gan function with the specified number of epochs and batch size
d_loss_list, g_loss_list, d_accuracy_list = train_gan(generator, discriminator, gan, x_train, y_train, epochs=num_epochs, batch_size=batch_size)


In [ ]:
import matplotlib.pyplot as plt

def plot_loss(d_loss, g_loss, num_epochs):
    plt.figure(figsize=(10, 5))
    plt.plot(range(num_epochs), d_loss, label="Discriminator Loss")
    plt.plot(range(num_epochs), g_loss, label="Generator Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

plot_loss(d_loss_list, g_loss_list, num_epochs)


#**Show results**

In [ ]:
def generate_colored_images(generator, gray_images):
    colored_images = generator.predict(gray_images)
    return colored_images

In [ ]:
num_examples_to_generate = len(x_test)
generated_images = generate_colored_images(generator, x_test)

def result_compare(y_test, pred, x_test):
    num_samples = len(y_test)

    for idx in range(num_samples):
        fig = plt.figure()
        ax1 = fig.add_subplot(1, 3, 1)
        ax1.imshow(x_test[idx], cmap='gray')
        ax1.set_title('Input')
        ax2 = fig.add_subplot(1, 3, 2)
        ax2.imshow(pred[idx])
        ax2.set_title('Output')
        ax3 = fig.add_subplot(1, 3, 3)
        ax3.imshow(y_test[idx])
        ax3.set_title('Original image')
        plt.show()

result_compare(y_test, generated_images, x_test)

#**Save model to deploy to website**

In [ ]:
from tensorflow.keras.models import load_model
gan.save('colorization_model.h5')
# generator.save('generator_model.h5')